In [1]:
import pyspark

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import isnan, when, count, col,slice
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
from numpy.random import rand
from pyspark.ml.feature import VectorSlicer
from pyspark.ml.feature import VectorAssembler,StringIndexer
from pyspark.ml import Pipeline

In [3]:
spark = SparkSession.builder.appName('app_name').getOrCreate()
#master('local[*]').config('spark.sql.execution.arrow.pyspark.enabled', True).config('spark.sql.session.timeZone', 'UTC').config('spark.driver.memory','32G').config('spark.ui.showConsoleProgress', True).config('spark.sql.repl.eagerEval.enabled', True)

In [ ]:
spark

In [5]:
#dataframe=spark.read.option('header','true','inferSchema','true').csv('02-14-2018.csv')

dataframe = spark.read.csv("02-14-2018.csv", header=True, inferSchema=True)

In [ ]:
dataframe.count()

In [6]:
dataframe.printSchema()

root
 |-- Dst Port: integer (nullable = true)
 |-- Protocol: integer (nullable = true)
 |-- Timestamp: string (nullable = true)
 |-- Flow Duration: long (nullable = true)
 |-- Tot Fwd Pkts: integer (nullable = true)
 |-- Tot Bwd Pkts: integer (nullable = true)
 |-- TotLen Fwd Pkts: integer (nullable = true)
 |-- TotLen Bwd Pkts: integer (nullable = true)
 |-- Fwd Pkt Len Max: integer (nullable = true)
 |-- Fwd Pkt Len Min: integer (nullable = true)
 |-- Fwd Pkt Len Mean: double (nullable = true)
 |-- Fwd Pkt Len Std: double (nullable = true)
 |-- Bwd Pkt Len Max: integer (nullable = true)
 |-- Bwd Pkt Len Min: integer (nullable = true)
 |-- Bwd Pkt Len Mean: double (nullable = true)
 |-- Bwd Pkt Len Std: double (nullable = true)
 |-- Flow Byts/s: double (nullable = true)
 |-- Flow Pkts/s: double (nullable = true)
 |-- Flow IAT Mean: double (nullable = true)
 |-- Flow IAT Std: double (nullable = true)
 |-- Flow IAT Max: long (nullable = true)
 |-- Flow IAT Min: long (nullable = true)
 |

In [7]:
dataframe = dataframe.dropna()
dataframe = dataframe.drop('Timestamp')

In [11]:
from pyspark.sql.functions import col
dataframe.select(*(col(c).cast("float").alias(c) for c in dataframe.columns))

DataFrame[Dst Port: float, Protocol: float, Flow Duration: float, Tot Fwd Pkts: float, Tot Bwd Pkts: float, TotLen Fwd Pkts: float, TotLen Bwd Pkts: float, Fwd Pkt Len Max: float, Fwd Pkt Len Min: float, Fwd Pkt Len Mean: float, Fwd Pkt Len Std: float, Bwd Pkt Len Max: float, Bwd Pkt Len Min: float, Bwd Pkt Len Mean: float, Bwd Pkt Len Std: float, Flow Byts/s: float, Flow Pkts/s: float, Flow IAT Mean: float, Flow IAT Std: float, Flow IAT Max: float, Flow IAT Min: float, Fwd IAT Tot: float, Fwd IAT Mean: float, Fwd IAT Std: float, Fwd IAT Max: float, Fwd IAT Min: float, Bwd IAT Tot: float, Bwd IAT Mean: float, Bwd IAT Std: float, Bwd IAT Max: float, Bwd IAT Min: float, Fwd PSH Flags: float, Bwd PSH Flags: float, Fwd URG Flags: float, Bwd URG Flags: float, Fwd Header Len: float, Bwd Header Len: float, Fwd Pkts/s: float, Bwd Pkts/s: float, Pkt Len Min: float, Pkt Len Max: float, Pkt Len Mean: float, Pkt Len Std: float, Pkt Len Var: float, FIN Flag Cnt: float, SYN Flag Cnt: float, RST Flag

In [8]:
dataframe.printSchema()

root
 |-- Dst Port: integer (nullable = true)
 |-- Protocol: integer (nullable = true)
 |-- Flow Duration: long (nullable = true)
 |-- Tot Fwd Pkts: integer (nullable = true)
 |-- Tot Bwd Pkts: integer (nullable = true)
 |-- TotLen Fwd Pkts: integer (nullable = true)
 |-- TotLen Bwd Pkts: integer (nullable = true)
 |-- Fwd Pkt Len Max: integer (nullable = true)
 |-- Fwd Pkt Len Min: integer (nullable = true)
 |-- Fwd Pkt Len Mean: double (nullable = true)
 |-- Fwd Pkt Len Std: double (nullable = true)
 |-- Bwd Pkt Len Max: integer (nullable = true)
 |-- Bwd Pkt Len Min: integer (nullable = true)
 |-- Bwd Pkt Len Mean: double (nullable = true)
 |-- Bwd Pkt Len Std: double (nullable = true)
 |-- Flow Byts/s: double (nullable = true)
 |-- Flow Pkts/s: double (nullable = true)
 |-- Flow IAT Mean: double (nullable = true)
 |-- Flow IAT Std: double (nullable = true)
 |-- Flow IAT Max: long (nullable = true)
 |-- Flow IAT Min: long (nullable = true)
 |-- Fwd IAT Tot: long (nullable = true)
 |

In [ ]:
# (train, test) = dataframe.randomSplit([0.8, 0.2])
# xtrain = train.select(train.iloc[:, :-1])
# ytrain = train.select(train.iloc[:-1])
# xtest = test.select(test.iloc[:, :-1])
# ytest = test.select(test.iloc[:,-1])


In [9]:
(train, test) = dataframe.randomSplit([0.8, 0.2])

In [27]:
len(train.columns)

79

In [15]:
xtrain = train.drop('Label')
print(xtrain)

DataFrame[Dst Port: int, Protocol: int, Flow Duration: bigint, Tot Fwd Pkts: int, Tot Bwd Pkts: int, TotLen Fwd Pkts: int, TotLen Bwd Pkts: int, Fwd Pkt Len Max: int, Fwd Pkt Len Min: int, Fwd Pkt Len Mean: double, Fwd Pkt Len Std: double, Bwd Pkt Len Max: int, Bwd Pkt Len Min: int, Bwd Pkt Len Mean: double, Bwd Pkt Len Std: double, Flow Byts/s: double, Flow Pkts/s: double, Flow IAT Mean: double, Flow IAT Std: double, Flow IAT Max: bigint, Flow IAT Min: bigint, Fwd IAT Tot: bigint, Fwd IAT Mean: double, Fwd IAT Std: double, Fwd IAT Max: bigint, Fwd IAT Min: bigint, Bwd IAT Tot: int, Bwd IAT Mean: double, Bwd IAT Std: double, Bwd IAT Max: int, Bwd IAT Min: int, Fwd PSH Flags: int, Bwd PSH Flags: int, Fwd URG Flags: int, Bwd URG Flags: int, Fwd Header Len: int, Bwd Header Len: int, Fwd Pkts/s: double, Bwd Pkts/s: double, Pkt Len Min: int, Pkt Len Max: int, Pkt Len Mean: double, Pkt Len Std: double, Pkt Len Var: double, FIN Flag Cnt: int, SYN Flag Cnt: int, RST Flag Cnt: int, PSH Flag Cnt

In [20]:
ytrain = train.select('Label')
ytrain.take(5)

[Row(Label='Benign'),
 Row(Label='Benign'),
 Row(Label='Benign'),
 Row(Label='Benign'),
 Row(Label='Benign')]

In [21]:
xtest = test.drop("Label")
ytest = test.select('Label')

In [22]:
# parameter
fold = {'xt':xtrain, 'yt':ytrain, 'xv':xtest, 'yv':ytest}
k    = 5     # k-value in KNN
N    = 10    # number of particles
T    = 100   # maximum number of iterations
opts = {'k':k, 'fold':fold, 'N':N, 'T':T}

In [23]:
#lb - lower bound, ub - upper bound, N - no. of particles, dim - no of columns
def init_position(lb, ub, N, dim):
    #creates an array of N rows and dim column matrix of type float with value 0.0
    X = np.zeros([N, dim], dtype='float')
    for i in range(N):
        for d in range(dim):
            #We add a random value to each element in matrix using initial population technique
            X[i,d] = lb[0,d] + (ub[0,d] - lb[0,d]) * rand()        
    return X

In [29]:
lb = 0
ub = 1
dim = len(xtrain.columns)
if np.size(lb) == 1:
    ub = ub * np.ones([1, dim], dtype='float')
    lb = lb * np.ones([1, dim], dtype='float')

print(dim)

78


In [31]:
#X- init array, thres - threshold to convert to 1 or 0
def binary_conversion(X, thres, N, dim):
    #creates new matrix
    Xbin = np.zeros([N, dim], dtype='int')
    for i in range(N):
        for d in range(dim):
            #depending on previous(init) matrix it will add 1 or 0 in Xbin
            if X[i,d] > thres:
                Xbin[i,d] = 1
            else:
                Xbin[i,d] = 0
    return Xbin

In [32]:
def boundary(x, lb, ub):
    if x < lb:
        x = lb
    if x > ub:
        x = ub   
    return x

In [37]:
def jfs(xtrain, ytrain, opts):
    # Parameters
    ub    = 1
    lb    = 0
    thres = 0.5
    b     = 1       # constant
    
    N        = opts['N'] #No of particles
    max_iter = opts['T']
    if 'b' in opts:
        b    = opts['b']
    
    # Dimension
    # returns the number of columns
    dim = len(xtrain.columns)
    if np.size(lb) == 1:
        ub = ub * np.ones([1, dim], dtype='float')
        lb = lb * np.ones([1, dim], dtype='float')
        
    # Initialize position 
    X    = init_position(lb, ub, N, dim)
    
    # Binary conversion
    Xbin = binary_conversion(X, thres, N, dim)
    
    # Fitness at first iteration
    fit  = np.zeros([N, 1], dtype='float')
    Xgb  = np.zeros([1, dim], dtype='float')
    fitG = float('inf')
    
    for i in range(N):
        fit[i,0] = Fun(xtrain, ytrain, Xbin[i,:], opts)
        if fit[i,0] < fitG:
            Xgb[0,:] = X[i,:]
            fitG     = fit[i,0]
        
    # Pre
    curve = np.zeros([1, max_iter], dtype='float') 
    t     = 0
    
    curve[0,t] = fitG.copy()
    print("Generation:", t + 1)
    print("Best (WOA):", curve[0,t])
    t += 1

    while t < max_iter:
        # Define a, linearly decreases from 2 to 0 
        a = 2 - t * (2 / max_iter)
        
        for i in range(N):
            # Parameter A (2.3)
            A = 2 * a * rand() - a
            # Paramater C (2.4)
            C = 2 * rand()
            # Parameter p, random number in [0,1]
            p = rand()
            # Parameter l, random number in [-1,1]
            l = -1 + 2 * rand()  
            # Whale position update (2.6)
            if p  < 0.5:
                # {1} Encircling prey
                if abs(A) < 1:
                    for d in range(dim):
                        # Compute D (2.1)
                        Dx     = abs(C * Xgb[0,d] - X[i,d])
                        # Position update (2.2)
                        X[i,d] = Xgb[0,d] - A * Dx
                        # Boundary
                        X[i,d] = boundary(X[i,d], lb[0,d], ub[0,d])
                
                # {2} Search for prey
                elif abs(A) >= 1:
                    for d in range(dim):
                        # Select a random whale
                        k      = np.random.randint(low = 0, high = N)
                        # Compute D (2.7)
                        Dx     = abs(C * X[k,d] - X[i,d])
                        # Position update (2.8)
                        X[i,d] = X[k,d] - A * Dx
                        # Boundary
                        X[i,d] = boundary(X[i,d], lb[0,d], ub[0,d])
            
            # {3} Bubble-net attacking 
            elif p >= 0.5:
                for d in range(dim):
                    # Distance of whale to prey
                    dist   = abs(Xgb[0,d] - X[i,d])
                    # Position update (2.5)
                    X[i,d] = dist * np.exp(b * l) * np.cos(2 * np.pi * l) + Xgb[0,d] 
                    # Boundary
                    X[i,d] = boundary(X[i,d], lb[0,d], ub[0,d])
        
        # Binary conversion
        Xbin = binary_conversion(X, thres, N, dim)
        
        # Fitness
        for i in range(N):
            fit[i,0] = Fun(xtrain, ytrain, Xbin[i,:], opts)
            if fit[i,0] < fitG:
                Xgb[0,:] = X[i,:]
                fitG     = fit[i,0]
        
        # Store result
        curve[0,t] = fitG.copy()
        print("Generation:", t + 1)
        print("Best (WOA):", curve[0,t])
        t += 1            

            
    # Best feature subset
    Gbin       = binary_conversion(Xgb, thres, 1, dim) 
    Gbin       = Gbin.reshape(dim)    
    pos        = np.asarray(range(0, dim))    
    sel_index  = pos[Gbin == 1]
    num_feat   = len(sel_index)
    # Create dictionary
    woa_data = {'sf': sel_index, 'c': curve, 'nf': num_feat}
    
    return woa_data

In [39]:
def Fun(xtrain, ytrain, x, opts):
    # Parameters
    alpha    = 0.99
    beta     = 1 - alpha
    # Original feature size
    max_feat = len(x)
    # Number of selected features
    num_feat = np.sum(x == 1)
    # Solve if no feature selected
    if num_feat == 0:
        cost  = 1
    else:
        # Get error rate
        error = error_rate(xtrain, ytrain, x, opts)
        # Objective function
        cost  = alpha * error + beta * (num_feat / max_feat)
        
    return cost

In [44]:
def error_rate(xtrain, ytrain, x, opts):
    # parameters
    k     = opts['k']
    fold  = opts['fold']
    xt    = fold['xt']
    yt    = fold['yt']
    xv    = fold['xv']
    yv    = fold['yv']
    
    # Number of instances
    num_train = len(xt.columns)
    num_valid = len(xv.columns)
    # Define selected features
    xtrain  = xt[:, x == 1]
    ytrain  = yt.reshape(num_train)  # Solve bug
    xvalid  = xv[:, x == 1]
    yvalid  = yv.reshape(num_valid)  # Solve bug   
    # Training
    mdl     = KNeighborsClassifier(n_neighbors = k)
    mdl.fit(xtrain, ytrain)
    # Prediction
    ypred   = mdl.predict(xvalid)
    acc     = np.sum(yvalid == ypred) / num_valid
    error   = 1 - acc
    
    return error

In [40]:
feat = dataframe.drop('Label')
label = dataframe.select("Label")

In [ ]:
# perform feature selection
fmdl = jfs(feat, label, opts)